In [1]:
from benchmark import sma_evaluation_short, mean_rev_evaluation
import pandas as pd
import numpy as np 
import utils
import labeling
import benchmark
import itertools
import warnings
import financial_eval

warnings.filterwarnings("ignore")

In [2]:
vgt_full = utils.data_loader(path="../DATA/VGT_historical data_2005.csv")
_, label_vgt, double_label = labeling.get_labels_daily(vgt_full, (1,1))
vgt_full["Y"] = label_vgt
vgt_full = vgt_full["2009-01-01":]
print(double_label)

train_size = int(0.8*len(vgt_full)) # in the preprocessing step we have to delete one row 
vgt_train = vgt_full[10:train_size]
vgt_test = vgt_full[train_size+10:]
vgt_train.head()

0.0095


,close,open,high,low,volume,change%,Y
date,,,,,,,
2009-01-16,33.47,33.68,33.84,32.64,141880.0,0.97,-1.0
2009-01-20,31.73,33.43,33.45,31.71,295440.0,-5.20,1.0
2009-01-21,33.31,32.28,33.35,32.00,308710.0,4.98,1.0
2009-01-22,32.59,32.53,34.55,32.00,391210.0,-2.16,0.0
2009-01-23,33.02,31.99,33.47,31.89,405150.0,1.32,0.0


In [5]:
xsd_full = utils.data_loader(path="../DATA/XSD Historical Data.csv")
_, label_xsd, double_label = labeling.get_labels_daily(xsd_full, (1,1))
xsd_full["Y"] = label_xsd
print(double_label)
xsd_full = xsd_full["2009-01-01":"2020-01-01"]

train_size = int(0.8*len(xsd_full)) 
xsd_train = xsd_full[10:train_size]
xsd_test = xsd_full[train_size+10:]
xsd_train.head()

0.0083


,close,open,high,low,volume,change%,Y
date,,,,,,,
2009-01-16,11.99,11.75,12.00,11.70,16250.0,4.35,-1.0
2009-01-20,11.18,11.63,11.66,11.18,1080.0,-6.76,0.0
2009-01-21,11.51,11.36,11.51,11.06,5800.0,2.95,-1.0
2009-01-22,11.26,11.21,11.38,11.00,16200.0,-2.17,1.0
2009-01-23,11.66,11.08,11.79,11.08,23650.0,3.55,0.0


In [6]:
results = pd.DataFrame(columns = ["Buy&Hold", "SMA Strategy", "SMA Parameter", "Mean-Reversion Strategy", "MR Parameter"], index=["VGT train", "VGT test", "XSD train", "XSD test"])

# Simulation of only Hold & Random Simulation

## Random Simulation VGT


In [8]:
from sklearn.metrics import accuracy_score, f1_score
accuracy_list = []
f1_list_weighted = []
f1_list_macro = []

y_test= vgt_test["Y"]
_, weights = np.unique(vgt_train["Y"], return_counts=True)/np.sum(np.unique(vgt_train["Y"], return_counts=True)[1])

for i in range(10000):
    y_random = np.random.choice([-1,0,1] , size= len(y_test), p= weights)
    accuracy = accuracy_score(y_test, y_random)
    f1_score_weighted = f1_score(y_test, y_random, average='weighted')
    f1_score_macro = f1_score(y_test, y_random, average='macro')
    accuracy_list.append(accuracy)
    f1_list_weighted.append(f1_score_weighted)
    f1_list_macro.append(f1_score_macro)
print(f"Average accuracy: {np.round(np.mean(accuracy_list),4)}")
print(f"Average F1-Score weighted: {np.round(np.mean(f1_list_weighted),4)}")
print(f"Average F1-Score macro: {np.round(np.mean(f1_list_macro),4)}")

Average accuracy: 0.3737
Average F1-Score weighted: 0.3747
Average F1-Score macro: 0.3331


## Random Simulation XSD

In [9]:
accuracy_list = []
f1_list_weighted = []
f1_list_macro = []
y_test= xsd_test["Y"]
_, weights = np.unique(xsd_train["Y"], return_counts=True)/np.sum(np.unique(xsd_train["Y"], return_counts=True)[1])

for i in range(10000):
    y_random = np.random.choice([-1,0,1] , size= len(y_test), p= weights)
    accuracy = accuracy_score(y_test, y_random)
    f1_score_weighted = f1_score(y_test, y_random, average='weighted')
    f1_score_macro = f1_score(y_test, y_random, average='macro')
    accuracy_list.append(accuracy)
    f1_list_weighted.append(f1_score_weighted)
    f1_list_macro.append(f1_score_macro)
print(f"Average accuracy: {np.round(np.mean(accuracy_list),4)}")
print(f"Average F1-Score weighted: {np.round(np.mean(f1_list_weighted),4)}")
print(f"Average F1-Score macro: {np.round(np.mean(f1_list_macro),4)}")

Average accuracy: 0.3712
Average F1-Score weighted: 0.3711
Average F1-Score macro: 0.3329


## Hold only Simulation VGT 

In [10]:
from sklearn.metrics import classification_report
hold_only = np.zeros(len(vgt_test["Y"]))
print(classification_report(vgt_test["Y"], hold_only, digits=4))

              precision    recall  f1-score   support

        -1.0     0.0000    0.0000    0.0000       128
         0.0     0.5018    1.0000    0.6683       273
         1.0     0.0000    0.0000    0.0000       143

    accuracy                         0.5018       544
   macro avg     0.1673    0.3333    0.2228       544
weighted avg     0.2518    0.5018    0.3354       544



## Hold only Simulation XSD 


In [11]:
hold_only = np.zeros(len(xsd_test["Y"]))
print(classification_report(xsd_test["Y"], hold_only, digits=4))

              precision    recall  f1-score   support

        -1.0     0.0000    0.0000    0.0000       133
         0.0     0.4926    1.0000    0.6601       268
         1.0     0.0000    0.0000    0.0000       143

    accuracy                         0.4926       544
   macro avg     0.1642    0.3333    0.2200       544
weighted avg     0.2427    0.4926    0.3252       544



# Buy & Hold

## VGT

In [12]:
return_train, capital_train =  financial_eval.buy_hold_evaluation(vgt_train)

End capital: 46868.84 euros
Total return through Buy & Hold: 369.0%


In [13]:
return_train, capital_train =  financial_eval.buy_hold_evaluation(vgt_train)
results["Buy&Hold"]["VGT train"] = str(round(return_train*100,4)) + ("%")

End capital: 46868.84 euros
Total return through Buy & Hold: 369.0%


In [14]:
return_test, capital_test =  financial_eval.buy_hold_evaluation(vgt_test)
results["Buy&Hold"]["VGT test"] =   str(round(return_test*100,4)) + ("%")

End capital: 15005.82 euros
Total return through Buy & Hold: 50.0%


## XSD

In [15]:
return_train, capital_train =  financial_eval.buy_hold_evaluation(xsd_train)
results["Buy&Hold"]["XSD train"] = str(round(return_train*100,4)) + ("%")

End capital: 56105.09 euros
Total return through Buy & Hold: 461.00000000000006%


In [16]:
return_test, capital_test =  financial_eval.buy_hold_evaluation(xsd_test)
results["Buy&Hold"]["XSD test"] =   str(round(return_test*100,4)) + ("%")

End capital: 15056.16 euros
Total return through Buy & Hold: 51.0%


# SMA Strategy Performance evaluation

In [17]:
grid={"sms_s_list": [10,15,20,25,30,35,40], "sms_l_list": [50,60,70,80,90,100]}

keys, values = zip(*grid.items())
permut_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]

## VGT

In [18]:
best_total_return = 0 
best_total_final_capital = 0

for i in range(len(permut_grid)):
    sms_s, sms_l = permut_grid[i].values()
    total_return, total_final_capital = sma_evaluation_short(vgt_train, sms_s, sms_l)
    
    if total_return > best_total_return: 
        best_total_final_capital = total_final_capital
        best_total_return= total_return 
        best_vals = {"best_sms_s": sms_s, "best_sms_l": sms_l}

Day 49: purchase of 288.62 units for total of 9995 euros

Day 259:
Total sale:346.34 units for total of 17845.36 euros
Closed long position: 288.62 units
New short position: 57.72 units

Day 286:
Short position closed: repurchase of 57.72 shares.
Long position built: 267.24 units. Total value: 14671.54 euros

Day 330:
Total sale:320.69 units for total of 17738.78 euros
Closed long position: 267.24 units
New short position: 53.45 units

Day 380:
Short position closed: repurchase of 53.45 shares.
Long position built: 278.39 units. Total value: 14877.41 euros

Day 399:
Total sale:334.07 units for total of 17306.51 euros
Closed long position: 278.39 units
New short position: 55.68 units

Day 419:
Short position closed: repurchase of 55.68 shares.
Long position built: 265.25 units. Total value: 14299.8 euros

Day 541:
Total sale:318.3 units for total of 20191.14 euros
Closed long position: 265.25 units
New short position: 53.05 units

Day 573:
Short position closed: repurchase of 53.05 shar

In [19]:
best_vals

{'best_sms_s': 40, 'best_sms_l': 90}

In [20]:
best_total_final_capital

24610.267734669374

In [21]:
best_total_return

1.4610267734669375

In [22]:
results["SMA Strategy"]["VGT train"] = str(round(best_total_return*100, 2))+"%"
results["SMA Parameter"]["VGT train"] = str("SMA short: " + str(best_vals["best_sms_s"]) +", SMA long: " +str(best_vals["best_sms_l"]))
results["SMA Parameter"]["VGT test"] = str("SMA short: " + str(best_vals["best_sms_s"]) +", SMA long: " +str(best_vals["best_sms_l"]))

In [23]:
return_test, _=sma_evaluation_short(vgt_test, best_vals["best_sms_s"], best_vals["best_sms_l"])

Day 89: purchase of 54.7 units for total of 9995 euros

Day 128:
Total sale:65.64 units for total of 11750.47 euros
Closed long position: 54.7 units
New short position: 10.94 units

Day 140:
Short position closed: repurchase of 10.94 shares.
Long position built: 53.3 units. Total value: 9745.09 euros

Day 256:
Total sale:63.96 units for total of 12179.38 euros
Closed long position: 53.3 units
New short position: 10.66 units

Day 330:
Short position closed: repurchase of 10.66 shares.
Long position built: 52.1 units. Total value: 10106.55 euros

End of observation period
Short position of 0 units closed for 0.0 euros.

End capital on day 544: 12756.93 euros
Total return: 27.57%
Shares held at end of period: 52.1
Total fee spending: 25


In [24]:
results["SMA Strategy"]["VGT test"] = str(round(return_test*100, 2))+"%"

## XSD

In [25]:
best_total_return = -10000
best_total_final_capital = 0

for i in range(len(permut_grid)):
    sms_s, sms_l = permut_grid[i].values()
    total_return, total_final_capital = sma_evaluation_short(xsd_train, sms_s, sms_l)
    
    if total_return > best_total_return: 
        best_total_final_capital = total_final_capital
        best_total_return= total_return 
        best_vals = {"best_sms_s": sms_s, "best_sms_l": sms_l}

Day 49: purchase of 731.7 units for total of 9995 euros

Day 197:
Total sale:878.04 units for total of 16493.37 euros
Closed long position: 731.7 units
New short position: 146.34 units

Day 212:
Short position closed: repurchase of 146.34 shares.
Long position built: 636.96 units. Total value: 13407.91 euros

Day 260:
Total sale:764.35 units for total of 16076.92 euros
Closed long position: 636.96 units
New short position: 127.39 units

Day 278:
Short position closed: repurchase of 127.39 shares.
Long position built: 573.52 units. Total value: 13150.87 euros

Day 329:
Total sale:688.22 units for total of 16415.93 euros
Closed long position: 573.52 units
New short position: 114.7 units

Day 380:
Short position closed: repurchase of 114.7 shares.
Long position built: 604.45 units. Total value: 13793.48 euros

Day 393:
Total sale:725.34 units for total of 15894.45 euros
Closed long position: 604.45 units
New short position: 120.89 units

Day 425:
Short position closed: repurchase of 120.8

In [26]:
best_vals

{'best_sms_s': 20, 'best_sms_l': 90}

In [27]:
best_total_final_capital

27896.12499275136

In [28]:
best_total_return

1.7896124992751359

In [29]:
results["SMA Strategy"]["XSD train"] = str(round(best_total_return*100, 2))+"%"
results["SMA Parameter"]["XSD train"] = str("SMA short: " + str(best_vals["best_sms_s"]) +", SMA long: " +str(best_vals["best_sms_l"]))
results["SMA Parameter"]["XSD test"] = str("SMA short: " + str(best_vals["best_sms_s"]) +", SMA long: " +str(best_vals["best_sms_l"]))

In [30]:
return_test, _ = sma_evaluation_short(xsd_test, best_vals["best_sms_s"], best_vals["best_sms_l"])

Day 90: purchase of 134.25 units for total of 9995 euros

Day 108:
Total sale:161.1 units for total of 11301.0 euros
Closed long position: 134.25 units
New short position: 26.85 units

Day 139:
Short position closed: repurchase of 26.85 shares.
Long position built: 125.9 units. Total value: 9310.44 euros

Day 235:
Total sale:151.08 units for total of 10717.15 euros
Closed long position: 125.9 units
New short position: 25.18 units

Day 314:
Short position closed: repurchase of 25.18 shares.
Long position built: 121.42 units. Total value: 8872.24 euros

Day 395:
Total sale:145.7 units for total of 10722.89 euros
Closed long position: 121.42 units
New short position: 24.28 units

Day 417:
Short position closed: repurchase of 24.28 shares.
Long position built: 100.72 units. Total value: 8636.12 euros

End of observation period
Short position of 0 units closed for 0.0 euros.

End capital on day 544: 10665.72 euros
Total return: 6.66%
Shares held at end of period: 100.72
Total fee spending: 

In [31]:
results["SMA Strategy"]["XSD test"] = str(round(return_test*100, 2))+"%"

# Mean-Reversion Strategy Performance Evaluation

In [32]:
grid={"sma_list": list(range(10,85,5)), "std_list": [0.5,1,1.5,2,2.5,3,3.5,4]}

keys, values = zip(*grid.items())
permut_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]

## VGT

In [33]:
best_total_return = -1000
best_total_final_capital = 0
for i in range(len(permut_grid)):
    sma, std_dev = permut_grid[i].values()
    total_return, total_final_capital = mean_rev_evaluation(sma=sma, std_dev=std_dev, data=vgt_train)
    if total_return > best_total_return: 
        best_total_final_capital = total_final_capital
        best_total_return= total_return
        best_vals = {"best_sma": sma, "best_std": std_dev}

Day 21: purchase of 298.36 units for total of 9995 euros

Day 35:
Went neutral. Long position closed - sold 298.36 units for 9294.83 euros.

Day 78: purchase of 235.3 units for total of 9289.83 euros

Day 83:
Went neutral. Long position closed - sold 235.3 units for 9353.06 euros.

Day 103: purchase of 223.42 units for total of 9348.06 euros

Day 110:
Went neutral. Long position closed - sold 223.42 units for 9497.23 euros.

Day 116: purchase of 227.85 units for total of 9492.23 euros

Day 121:
Went neutral. Long position closed - sold 227.85 units for 9512.29 euros.

Day 142: purchase of 205.65 units for total of 9507.29 euros

Day 143:
Went neutral. Long position closed - sold 205.65 units for 9646.25 euros.

Day 147: purchase of 208.05 units for total of 9641.25 euros

Day 148:
Went neutral. Long position closed - sold 208.05 units for 9692.42 euros.

Day 158: purchase of 207.48 units for total of 9687.42 euros

Day 160:
Went neutral. Long position closed - sold 207.48 units for 992

In [34]:
best_total_final_capital

24359.86426904214

In [35]:
best_total_return

1.435986426904214

In [36]:
best_vals

{'best_sma': 10, 'best_std': 1}

In [37]:
results["Mean-Reversion Strategy"]["VGT train"] = str(round(best_total_return*100, 2))+"%"
results["MR Parameter"]["VGT train"] = str("SMA: " + str(best_vals["best_sma"]) +", STD: " +str(best_vals["best_std"]))
results["MR Parameter"]["VGT test"] = str("SMA: " + str(best_vals["best_sma"]) +", STD: " +str(best_vals["best_std"]))

In [38]:
return_test, _ = mean_rev_evaluation(sma=best_vals["best_sma"], std_dev=best_vals["best_std"], data=vgt_test)

Day 10: purchase of 61.23 units for total of 9995 euros

Day 11:
Went neutral. Long position closed - sold 61.23 units for 10127.77 euros.

Day 21: purchase of 61.69 units for total of 10122.77 euros

Day 26:
Went neutral. Long position closed - sold 61.69 units for 10117.77 euros.

Day 40: purchase of 61.39 units for total of 10112.77 euros

Day 41:
Went neutral. Long position closed - sold 61.39 units for 10245.9 euros.

Day 63: purchase of 59.52 units for total of 10240.9 euros

Day 71:
Went neutral. Long position closed - sold 59.52 units for 10201.38 euros.

Day 94: purchase of 57.02 units for total of 10196.38 euros

Day 105:
Went neutral. Long position closed - sold 57.02 units for 9765.41 euros.

Day 118: purchase of 57.68 units for total of 9760.41 euros

Day 123:
Went neutral. Long position closed - sold 57.68 units for 9975.18 euros.

Day 159: purchase of 53.36 units for total of 9970.18 euros

Day 168:
Went neutral. Long position closed - sold 53.36 units for 9793.38 euros.

In [39]:
results["Mean-Reversion Strategy"]["VGT test"] = str(round(return_test*100, 2))+"%"

## XSD

In [40]:
best_total_return = 0 
best_total_final_capital = 0
for i in range(len(permut_grid)):
    sma, std_dev = permut_grid[i].values()
    total_return, total_final_capital = mean_rev_evaluation(sma=sma, std_dev=std_dev, data=xsd_train)
    if total_return > best_total_return: 
        best_total_final_capital = total_final_capital
        best_total_return= total_return
        best_vals = {"best_sma": sma, "best_std": std_dev}

Day 21: purchase of 819.26 units for total of 9995 euros

Day 26:
Went neutral. Long position closed - sold 819.26 units for 9916.27 euros.

Day 34: purchase of 863.35 units for total of 9911.27 euros

Day 35:
Went neutral. Long position closed - sold 863.35 units for 10752.35 euros.

Day 50: purchase of 785.63 units for total of 10747.35 euros

Day 51:
Went neutral. Long position closed - sold 785.63 units for 11033.03 euros.

Day 64: purchase of 747.16 units for total of 11028.03 euros

Day 65:
Went neutral. Long position closed - sold 747.16 units for 11284.54 euros.

Day 77: purchase of 747.48 units for total of 11279.54 euros

Day 84:
Went neutral. Long position closed - sold 747.48 units for 11648.28 euros.

Day 105: purchase of 687.32 units for total of 11643.28 euros

Day 110:
Went neutral. Long position closed - sold 687.32 units for 11679.52 euros.

Day 117: purchase of 721.54 units for total of 11674.52 euros

Day 121:
Went neutral. Long position closed - sold 721.54 units f

In [41]:
best_total_final_capital

32600.436170935784

In [42]:
best_total_return

2.2600436170935785

In [43]:
best_vals

{'best_sma': 10, 'best_std': 0.5}

In [44]:
results["Mean-Reversion Strategy"]["XSD train"] = str(round(best_total_return*100, 2))+"%"
results["MR Parameter"]["XSD train"] = str("SMA: " + str(best_vals["best_sma"]) +", STD: " +str(best_vals["best_std"]))
results["MR Parameter"]["XSD test"] = str("SMA: " + str(best_vals["best_sma"]) +", STD: " +str(best_vals["best_std"]))

In [45]:
return_test, _ = mean_rev_evaluation(sma=best_vals["best_sma"], std_dev=best_vals["best_std"], data=xsd_test)

Day 20: purchase of 140.42 units for total of 9995 euros

Day 30:
Went neutral. Long position closed - sold 140.42 units for 9734.44 euros.

Day 40: purchase of 139.37 units for total of 9729.44 euros

Day 41:
Went neutral. Long position closed - sold 139.37 units for 10036.63 euros.

Day 57: purchase of 138.1 units for total of 10031.63 euros

Day 58:
Went neutral. Long position closed - sold 138.1 units for 10302.83 euros.

Day 61: purchase of 140.3 units for total of 10297.83 euros

Day 71:
Went neutral. Long position closed - sold 140.3 units for 9598.36 euros.

Day 96: purchase of 132.49 units for total of 9593.36 euros

Day 108:
Went neutral. Long position closed - sold 132.49 units for 9292.91 euros.

Day 116: purchase of 133.83 units for total of 9287.91 euros

Day 124:
Went neutral. Long position closed - sold 133.83 units for 9189.23 euros.

Day 159: purchase of 121.2 units for total of 9184.23 euros

Day 168:
Went neutral. Long position closed - sold 121.2 units for 8985.32 

In [46]:
results["Mean-Reversion Strategy"]["XSD test"] = str(round(return_test*100, 2))+"%"

# All results 

In [47]:
results 

,Buy&Hold,SMA Strategy,SMA Parameter,Mean-Reversion Strategy,MR Parameter
VGT train,369.0%,146.1%,"SMA short: 40, SMA long: 90",143.6%,"SMA: 10, STD: 1"
VGT test,50.0%,27.57%,"SMA short: 40, SMA long: 90",-8.17%,"SMA: 10, STD: 1"
XSD train,461.0%,178.96%,"SMA short: 20, SMA long: 90",226.0%,"SMA: 10, STD: 0.5"
XSD test,51.0%,6.66%,"SMA short: 20, SMA long: 90",6.1%,"SMA: 10, STD: 0.5"


----